Keras 函数式 API 是定义复杂模型（如多输出模型、有向无环图或具有共享层的模型）的方法。

### 例一：全连接网络

Sequential 模型可能是实现这种网络的一个更好选择，但这个例子能够有助于开发的时候理解。
1. 网络层的实例是可调用的，它以张量为参数，并且返回一个张量
2. 输入和输出均为张量，它们都可以用来定义一个模型（Model）
3. 这样的模型同 Keras 的 Sequential 模型一样，都可以被训练

In [ ]:
import keras
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
# 定义一个张量
inputs = Input(shape=(512,))

In [ ]:
# 层的实例是可调用的，它以张量为参数，并且返回一个张量
output_1 = Dense(64,activation='relu')(inputs)
output_2 = Dense(64,activation='relu')(output_1)
predictions = Dense(1,activation = 'softmax')(output_2)

In [ ]:
# 创建一个包含输入层和三个全连接层的模型
model = Model(inputs=inputs,outputs=predictions)
model.compile(optimizer='rmsprop',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

数据测试了一下还有问题

In [28]:
# # 生成虚拟数据
# import numpy as np
# # 生成100行50列的随机浮点数，浮点数范围：（0，1）
# data = np.random.random((1000,512))
# print(data)
# # 1000行一列的随机整数，从[0,2)中随机。
# labels = np.random.randint(2,size=(1000,1))
# # 开始训练
# model.fit(data,labels)

### 所有的模型都可调用，就像网络层一样
利用函数式 API，可以轻易地重用训练好的模型：可以将任何模型看作是一个层，然后通过传递一个张量来调用它。注意，在调用模型时，不仅重用模型的结构，还重用了它的权重。

In [12]:
x = Input(shape=(784,))
# 这是可行的，并且返回上面定义的 10-way softmax。
y = model(x)

这种方式能允许我们快速创建可以处理序列输入的模型。只需一行代码，你就将图像分类模型转换为视频分类模型。

In [30]:
from keras.layers import TimeDistributed

# 输入张量是 20 个时间步的序列，
# 每一个时间为一个 784 维的向量
input_sequences = Input(shape=(20, 784))

In [ ]:
# 这部分将我们之前定义的模型应用于输入序列中的每个时间步。
# 之前定义的模型的输出是一个 10-way softmax，
# 因而下面的层的输出将是维度为 10 的 20 个向量的序列。
# processed_sequences = TimeDistributed(model)(input_sequences)

### 多输入多输出模型
以下是函数式 API 的一个很好的例子：具有多个输入和输出的模型。函数式 API 使处理大量交织的数据流变得容易。

![title](img/multi-input-multi-output-graph.png)

### 函数式 API 来实现
主要输入接收新闻标题本身，即一个整数序列（每个整数编码一个词）。
这些整数在 1 到 10,000 之间（10,000 个词的词汇表），且序列长度为 100 个词

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
import numpy as np
np.random.seed(0)  # 设置随机种子，用于复现结果

In [ ]:
# 标题输入：接收一个含有 100 个整数的序列，每个整数在 1 到 10000 之间。
# 注意我们可以通过传递一个 "name" 参数来命名任何层。
main_input = Input(shape=(100,),dtype='int32',name='main_input')

# Embedding 层将输入序列编码为一个稠密向量的序列，
# 每个向量维度为 512。
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# LSTM 层把向量序列转换成单个向量，
# 它包含整个序列的上下文信息
lstm_out = LSTM(32)(x)

In [ ]:
# 插入辅助损失，使得即使在模型主损失很高的情况下，LSTM 层和 Embedding 层都能被平稳地训练。
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

svd论文中也使用了输出连接的语法
![title](img/multi-input-multi-output-graph.png)

In [ ]:
# 将辅助输入数据与 LSTM 层的输出连接起来，输入到模型中
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# 堆叠多个全连接网络层
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# 最后添加主要的逻辑回归层
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

#### 然后定义一个具有两个输入和两个输出的模型

In [ ]:
model = Model(inputs=[main_input,auxiliary_input],outputs=[main_output,auxiliary_output])

编译模型，并给辅助损失分配一个 0.2 的权重。如果要为不同的输出指定不同的 loss_weights 或 loss，可以使用列表或字典。 在这里，我们给 loss 参数传递单个损失函数，这个损失将用于所有的输出

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])

In [ ]:
# 通过传递输入数组和目标数组的列表来训练模型
headline_data = np.round(np.abs(np.random.rand(12, 100) * 100))
additional_data = np.random.randn(12, 5)
headline_labels = np.random.randn(12, 1)
additional_labels = np.random.randn(12, 1)
model.fit([headline_data, additional_data], [headline_labels, additional_labels],
          epochs=50, batch_size=32)

In [ ]:
# 由于输入和输出均被命名了（在定义时传递了一个 name 参数），我们也可以通过以下方式编译模型：
# model.compile(optimizer='rmsprop',
#               loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
#               loss_weights={'main_output': 1., 'aux_output': 0.2})

# # 然后使用以下方式训练：
# model.fit({'main_input': headline_data, 'aux_input': additional_data},
#           {'main_output': headline_labels, 'aux_output': additional_labels},
#           epochs=50, batch_size=32)

In [ ]:
# 若使用此模型做推理，可以
model.predict({'main_input': headline_data, 'aux_input': additional_data})
# 或者
# pred = model.predict([headline_data, additional_data])

### 共享网络层

In [ ]:
import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

tweet_a = Input(shape=(280, 256))
tweet_b = Input(shape=(280, 256))

要在不同的输入上共享同一个层，只需实例化该层一次，然后根据需要传入你想要的输入即可

In [ ]:
# 这一层可以输入一个矩阵，并返回一个 64 维的向量 该层将被重用
shared_lstm = LSTM(64)

In [ ]:
# 当我们重用相同的图层实例多次，图层的权重也会被重用 (它其实就是同一层)
encoded_a = shared_lstm(tweet_a)
encoded_b = shared_lstm(tweet_b)

# 然后再连接两个向量：
merged_vector = keras.layers.concatenate([encoded_a, encoded_b], axis=-1)


In [ ]:
# 再在上面添加一个逻辑回归层
predictions = Dense(1, activation='sigmoid')(merged_vector)

# 定义一个连接推特输入和预测的可训练的模型
model = Model(inputs=[tweet_a, tweet_b], outputs=predictions)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit([data_a, data_b], 
          labels, 
          epochs=10)

### 层「节点」的概念

在某个输入上调用一个层时，都将创建一个新的张量（层的输出），并且为该层添加一个「节点」，将输入张量连接到输出张量。当多次调用同一个图层时，该图层将拥有多个节点索引 (0, 1, 2...)。

在之前版本的 Keras 中，可以通过 layer.get_output() 来获得层实例的输出张量，或者通过 layer.output_shape 来获取其输出形状。现在你依然可以这么做（除了 get_output() 已经被 output 属性替代）。但是如果一个层与多个输入连接

In [ ]:
a = Input(shape=(280, 256))

lstm = LSTM(32)
encoded_a = lstm(a)

assert lstm.output == encoded_a

In [ ]:
#  出现错误
a = Input(shape=(280, 256))
b = Input(shape=(280, 256))

lstm = LSTM(32)
encoded_a = lstm(a)
encoded_b = lstm(b)

lstm.get_output()

# >> AttributeError: Layer lstm_1 has multiple inbound nodes,
# hence the notion of "layer output" is ill-defined.
# Use `get_output_at(node_index)` instead.

In [ ]:
# 解决办法
assert lstm.get_output_at(0) == encoded_a
assert lstm.get_output_at(1) == encoded_b

#### input_shape 和 output_shape 这两个属性也是如此：只要该层只有一个节点，或者只要所有节点具有相同的输入/输出尺寸，那么「层输出/输入尺寸」的概念就被很好地定义，且将由 layer.output_shape / layer.input_shape 返回。但是比如说，如果将一个 Conv2D 层先应用于尺寸为 (32，32，3) 的输入，再应用于尺寸为 (64, 64, 3) 的输入，那么这个层就会有多个输入/输出尺寸，你将不得不通过指定它们所属节点的索引来获取它们·

In [ ]:
a = Input(shape=(32, 32, 3))
b = Input(shape=(64, 64, 3))

conv = Conv2D(16, (3, 3), padding='same')
conved_a = conv(a)

# 到目前为止只有一个输入，以下可行：
assert conv.input_shape == (None, 32, 32, 3)

In [ ]:
conved_b = conv(b)
# 现在 `.input_shape` 属性不可行，但是这样可以：
assert conv.get_input_shape_at(0) == (None, 32, 32, 3)
assert conv.get_input_shape_at(1) == (None, 64, 64, 3)

In [ ]:
from keras.models import load_model

model.save('my_model.h5')  # 创建 HDF5 文件 'my_model.h5'